From: https://github.com/Quorumetrix/Blender_scripts/blob/main/Mesh%20Decimation%20Pipeline.ipynb

In [2]:
import pandas as pd
import numpy as np
import os
import time
import h5py
import pyvista as pv

# Set up paths and parameters
obj_path = 'data/cell_meshes/'
dec_prcnt = 95
dec_path = 'data/cell_meshes/dec/'

# Load cell cellids into a list
cell_list = [864691136194086870]

#[864691134964446239, 864691135012524790, 864691135256138671, 864691135269913253, 864691135337851366, 864691135341421745, 864691135348239831, 864691135386363265, 864691135497750291, 864691135657783170, 864691135809446092, 864691135837182867, 864691135880405261, 864691136023933241, 864691136194008022, 864691136330394007]

def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as f:
        vertices = np.array(f['vertices'])
        faces = np.array(f['faces'])
    return vertices, faces

def create_pyvista_mesh(vertices, faces):
    # PyVista expects faces to be in the format (n, v1, v2, v3, ...)
    faces_with_size = np.hstack([np.full((faces.shape[0], 1), faces.shape[1]), faces])
    return pv.PolyData(vertices, faces_with_size)

def process_obj(cellid, dec_prcnt, obj_path, dec_path):
    start_time = time.time()
    
    # Create the full path to the file
    file_path = os.path.join(obj_path, str(cellid) + '_0.h5')
    print(f"Loading {file_path}")
    
    # Load the file
    vertices, faces = read_h5_file(file_path)
    load_time = time.time()
    print(f"Loaded {file_path} in {load_time - start_time:.2f} seconds")
    
    # Create PyVista mesh
    mesh = create_pyvista_mesh(vertices, faces)
    print(f"Original mesh info: {mesh}")
    
    # Perform decimation
    print(f"Decimating {file_path} by {dec_prcnt}%")
    simplified_mesh = mesh.decimate(target_reduction=dec_prcnt / 100)  # Reduce to x% of the original mesh
    decimate_time = time.time()
    print(f"Decimated {file_path} in {decimate_time - load_time:.2f} seconds")
    
    # Print simplified mesh information
    print(f"Simplified mesh info: {simplified_mesh}")
    
    # Ensure the output directory exists
    if not os.path.exists(dec_path):
        os.makedirs(dec_path)
    
    # Save the simplified mesh in PLY format
    output_filename = os.path.basename(file_path).replace('.h5', '.ply')
    output_path = os.path.join(dec_path, output_filename)
    
    try:
        simplified_mesh.save(output_path)
        save_time = time.time()
        print(f"Saved {output_path} in {save_time - decimate_time:.2f} seconds")
    except Exception as e:
        print(f"Error saving {output_path}: {e}")
    
    total_time = time.time() - start_time
    print(f"Total time: {total_time // 60} minutes and {total_time % 60:.2f} seconds")

# Process each cellid in the cell_list
for cellid in cell_list:
    process_obj(cellid, dec_prcnt, obj_path, dec_path)


Loading data/cell_meshes/864691136194086870_0.h5
Loaded data/cell_meshes/864691136194086870_0.h5 in 1.77 seconds
Original mesh info: PolyData (0x19f0a149d08)
  N Cells:	15867868
  N Points:	7894636
  N Strips:	0
  X Bounds:	6.589e+05, 7.175e+05
  Y Bounds:	4.819e+05, 5.400e+05
  Z Bounds:	7.868e+05, 8.425e+05
  N Arrays:	0

Decimating data/cell_meshes/864691136194086870_0.h5 by 95%
Decimated data/cell_meshes/864691136194086870_0.h5 in 231.50 seconds
Simplified mesh info: PolyData (0x19f0a149ee8)
  N Cells:	793390
  N Points:	387890
  N Strips:	0
  X Bounds:	6.591e+05, 7.173e+05
  Y Bounds:	4.819e+05, 5.400e+05
  Z Bounds:	7.869e+05, 8.424e+05
  N Arrays:	0

Saved data/cell_meshes/dec/864691136194086870_0.ply in 0.73 seconds
Total time: 3.0 minutes and 53.99 seconds
